In [2]:
import numpy as np
from scipy.stats import beta, bernoulli

Let each agent in our simulation have a feature which we will call `blue`. In this scenario, `blue` is a binary random variable that is either 0 or 1 probability $p$. 

That is, $X_1, \ldots, X_N$ is a random sample of $N$ Bernoulli random variables with parameter $p$, where $p$ is unkown. Let us assume that the prior distribution of $p$ is $p\sim B(\alpha_0, \beta_0)$ with prior probability density function $\xi$. If $x = (X_1 = x_1, \ldots, X_N = x_N)$ and the likelihood of observing $x$ given $p$ is $f(x_1|p)\cdots f(x_N|p) = f_N(x|p)$, then the posterior proabiity density function of $p$ after observing $x$ is
$$\xi(p|x) = \frac{f_N(x|p) * \xi(p)}{\int_0^1 f_N(x|p)\xi(p)dp}$$

We can find the updating rule for the posterior with a bit of math. Let $g_N(x) = \int_0^1 f_N(x|p)\xi(p)dp$ and consider, 
$$\xi(p|x) = \frac{f_N(x|p) * \xi(p)}{g_N(x)}$$
Evaluating $f_N(x|p)$ now,
$$f_N(x|p) = \prod_{i=1}^N p^{x_i}(1-p)^{x_i} = p^{\sum_{i=1}^Nx_i}(1-p)^{N - \sum_{i=1}^Nx_i} = p^y(1-p)^{N-y}$$
where $y = \sum_{i=1}^N x_i$. Evaluating $\xi(p)$ now, $$ \frac{\Gamma(\alpha_0 + \beta_0)}{\Gamma(\alpha_0)\Gamma(\beta_0)}p^{\alpha_0-1}(1-p)^{\beta_0-1}  $$